In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Any, Dict, List

In [3]:
from agorama.state_agent.state_agent import BasePerceptionAgent, action

In [3]:
from litellm import acompletion
from pydantic import BaseModel
import json


In [205]:
class FunctionParameter(BaseModel):
    type: str # the type of the parameter
    description: str # a description of what the parameter is
    required: bool # whether the parameter is required

class ToolClass(BaseModel):
    name: str
    description: str
    parameters: Dict[str, FunctionParameter]


    def payload(self):
        required_parameters = [param_name for param_name, param in self.parameters.items() if param.required]
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": {
                    "type": "object",
                    "properties": {
                        param_name: {
                            "type": param.type,
                            "description": param.description
                        }
                        for param_name, param in self.parameters.items()
                    },
                    "required": required_parameters
                }
            }
        }

In [161]:
from dataclasses import Field
from typing import Optional


class Item(BaseModel):
    """
    A base class for all items in the game.
    """
    name: str
    description: str
    quantity: int
    size: int
    functionality: str # a description of what the item can do

    def __str__(self) -> str:
        return f"{self.name} (x{self.quantity})\n" \
               f"Description: {self.description}\n" \
               f"Size: {self.size}\n" \
               f"Use: {self.functionality}"
    

class Inventory(BaseModel):
    items: List[Item]
    max_capacity: int
    remaining_capacity: Optional[int] = None

    def __post_init__(self):
        self.remaining_capacity = self.max_capacity - self._calculate_capacity()

    def _calculate_capacity(self):
        return sum(item.size for item in self.items)

    def update_inventory(self, items: List[Item]):
        self.items = items

    def __str__(self) -> str:
        if not self.items:
            return "Inventory is empty\nRemaining capacity: " + str(self.max_capacity)
        inventory_str = "Inventory Contents:\n"
        inventory_str += "=" * 20 + "\n"
        for item in self.items:
            inventory_str += str(item) + "\n"
            inventory_str += "-" * 20 + "\n"
        inventory_str += f"Remaining capacity: {self.remaining_capacity}"
        return inventory_str


In [162]:
class TextAdventure:
    def __init__(self, llm: str = "gpt-4o-mini", context_window: int = 1000):
        self.running_cost: float = 0.0

        self.system_prompt = """
        You are a text adventure game. Upon starting you should create an inventory for the player by specifying the capacity.
        You should always abide by the scenario provided, being reasonably realistic while mainting whimsy and fun.

        You will be given the entire chat history of your messages with the user.
        You will be given an Inventory object that you can use to add items to the player's inventory.
        You have three key functions you can call:
            - update_inventory(items: List[Item]) - Update the inventory (this replaces the entire inventory so you should manage it carefully)
            - noop() - Do nothing to the inventory. 

        For all three of your functions, the user will *still* recieve your chat response. 
        The inventory is managed for you to ensure that you never forget or hallucinate the items the player has access to.
        You can assume that the user can independently view the inventory, so you do not need to include it in your response.

        Keep it fun and engaging!

        # SCENARIO
        {scenario}

        # INVENTORY
        {inventory}
        """
        self.context_window = context_window
        self.messages = []

    def _format_system_prompt(self, scenario: str, inventory: Inventory):
        return self.system_prompt.format(scenario=scenario, inventory=inventory)
    
    async def user_chat(self, message: str):
        """

        """
        self.messages.append({"role": "user", "content": message})
        response = await acompletion(
            model=self.llm,
            messages=self.messages,
        )
        self.messages.append({"role": "assistant", "content": response.choices[0].message.content})
    
    async def setup(self, scenario: str):
        self.scenario = scenario
        self.inventory = self.setup_inventory(scenario)
        # now we get the first message to the user
        self.messages.append({"role": "system", "content": self._format_system_prompt(scenario, self.inventory)}) # system prompt
        initial_response = await acompletion(
            model=self.llm,
            messages = self.messages + [{"role": "user", "content": "Please start this adventure for me"}],
        )
        self.messages.append({"role": "assistant", "content": initial_response.choices[0].message.content})

    async def setup_inventory(self, scenario: str):
        """
        Setup the game with a scenario.
        """
        setup_response = await acompletion(
            model=self.llm,
            messages=[
                {"role": "system", "content": "You are setting up a text adventure game. Your job is to create a reasonable inventory for the player based on the scenario provided."},
                {"role": "user", "content": f"Given the scenario: {scenario}, create an inventory for the player."}
            ],
            response_format=Inventory
        )
        self.running_cost += setup_response._hidden_params['response_cost']
        try:
            inventory = Inventory(**json.loads(setup_response.choices[0].message.content))
            self.inventory = inventory
        except Exception as e:
            print(f"Error creating inventory: {e}")
            raise e
    
    def update_inventory(self, items: List[Item]):
        self.inventory.update_inventory(items)

In [163]:
textadventure = TextAdventure()

In [ ]:
await textadventure.setup_inventory("This is a cowboy western themed text adventure.")

14:36:04 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
14:36:08 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-2024-08-06
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-2024-08-06
14:36:08 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-2024-08-06


In [173]:
check = ToolClass(
    name="update_inventory",
    description="Update the inventory",
    parameters={
        "items": FunctionParameter(type="list[Item]", description="A list of Item objects to update the inventory with", required=True)
    },
    required_parameters=["items"],
)


In [174]:
check.payload()

{'type': 'function',
 'function': {'name': 'update_inventory',
  'description': 'Update the inventory',
  'parameters': {'type': 'object',
   'properties': {'items': {'type': 'list[Item]',
     'description': 'A list of Item objects to update the inventory with'}},
   'required': ['items']}}}

In [126]:
print(textadventure._format_system_prompt(scenario="You are in a room with a table and a chair.", inventory=Inventory(**json.loads(result.choices[0].message.content))))


        You are a text adventure game. Upon starting you should create an inventory for the player by specifying the capacity.
        You should always abide by the scenario provided, being reasonably realistic while mainting whimsy and fun.

        You will be given the entire chat history of your messages with the user.
        You will be given an Inventory object that you can use to add items to the player's inventory.
        You have three key functions you can call:
            - add_to_inventory(item: Item) - Add an item to the player's inventory
            - remove_from_inventory(item: Item) - Remove an item from the player's inventory
            - noop() - Do nothing to the inventory. 

        For all three of your functions, the user will *still* recieve your chat response. 
        The inventory is managed for you to ensure that you never forget or hallucinate the items the player has access to.

        Keep it fun and engaging!

        # SCENARIO
        You are in

In [134]:
result._hidden_params['response_cost']

7.365e-05

In [233]:
tc = ToolClass(
    name="noop",
    description="Do nothing",
    parameters={},
)

tc2 = ToolClass(
    name="Do_something", 
    description="Do something",
    parameters={
        "swag": FunctionParameter(type="string", description="This should be the string `swag`", required=True)
    },
)


In [234]:
tc2.payload()

{'type': 'function',
 'function': {'name': 'Do_something',
  'description': 'Do something',
  'parameters': {'type': 'object',
   'properties': {'swag': {'type': 'string',
     'description': 'This should be the string `swag`'}},
   'required': ['swag']}}}

In [235]:
result = await acompletion(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You should always choose to do something if given the option."}
    ],
    tools=[tc.payload(), tc2.payload()]
)

17:03:26 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:03:27 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


17:03:27 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


In [236]:
result.choices[0].message.tool_calls[0].function

Function(arguments='{"swag":"swag"}', name='Do_something')

In [298]:
class MemoryKeyAccess(BaseModel):
    """
    This is a helper class to store the memory key access information
    """
    key_list: List[str]

In [4]:
from agorama.models import ChatMessage
from agorama.state_agent.state_agent import ChatAgentWithMemory, MemoryKeyAccess

In [83]:
# setup for test

chat_agent = ChatAgentWithMemory(name="test", llm_model="gpt-4o-mini")
chat_agent.state['chat_history'].append(ChatMessage(message="Can you fetch the test_key for me and then respond", created_by="user"))
# chat_agent.state['chat_history'].append(ChatMessage(message="Actually scratch that, I don't want you to fetch any keys at all.", created_by="user"))
chat_agent.state['memory']['test_key'] = 'test_value'

In [84]:
result = chat_agent.perceive()

18:23:47 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:23:48 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:23:48 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


In [85]:
chat_agent.state['contextual_memory']

{'test_key': 'test_value'}

In [86]:
chat_agent.decide()

18:23:48 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:23:49 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:23:49 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


'respond'

In [87]:
chat_agent.act('respond')

18:23:49 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:23:50 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:23:50 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


'The test_key you asked for has the value: test_value. How can I assist you further?'

In [88]:
chat_agent.state['chat_history'].append(ChatMessage(message="Can you please delete the test_key from your memory?", created_by="user"))

In [89]:
chat_agent.perceive()

18:24:17 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:24:17 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:24:17 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


In [102]:
chat_agent.decide()

18:27:17 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:27:17 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:27:18 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


'remove'

In [103]:
chat_agent.state

{'memory': {'test_key': 'test_value'},
 'action_methods': [ToolClass(name='store', description='Store important information in memory', parameters={'key': FunctionParameter(type='string', description='The key to store the information under', required=True), 'value': FunctionParameter(type='string', description='The information to store', required=True)}),
  ToolClass(name='remove', description='Remove information from memory', parameters={'key': FunctionParameter(type='string', description='The key to remove the information from', required=True)}),
  ToolClass(name='respond', description='Send a message to the user', parameters={}),
  ToolClass(name='noop', description='Do nothing', parameters={})],
 'chat_history': [ChatMessage(message='Can you fetch the test_key for me and then respond', created_by='user', created_at=datetime.datetime(2025, 4, 21, 1, 23, 47, 225658, tzinfo=datetime.timezone.utc)),
  ChatMessage(message='The test_key you asked for has the value: test_value. How can I 

In [104]:
chat_agent.act('remove')

"Removed 'test_key' from memory"

In [106]:
chat_agent.state

{'memory': {},
 'action_methods': [ToolClass(name='store', description='Store important information in memory', parameters={'key': FunctionParameter(type='string', description='The key to store the information under', required=True), 'value': FunctionParameter(type='string', description='The information to store', required=True)}),
  ToolClass(name='remove', description='Remove information from memory', parameters={'key': FunctionParameter(type='string', description='The key to remove the information from', required=True)}),
  ToolClass(name='respond', description='Send a message to the user', parameters={}),
  ToolClass(name='noop', description='Do nothing', parameters={})],
 'chat_history': [ChatMessage(message='Can you fetch the test_key for me and then respond', created_by='user', created_at=datetime.datetime(2025, 4, 21, 1, 23, 47, 225658, tzinfo=datetime.timezone.utc)),
  ChatMessage(message='The test_key you asked for has the value: test_value. How can I assist you further?', cr

In [107]:
chat_agent.state['chat_history'].append(
    ChatMessage(message="Can you please add a new memory that says that I am a Founding CEO of a company called Agorama", created_by="user")
)

In [109]:
chat_agent.state

{'memory': {},
 'action_methods': [ToolClass(name='store', description='Store important information in memory', parameters={'key': FunctionParameter(type='string', description='The key to store the information under', required=True), 'value': FunctionParameter(type='string', description='The information to store', required=True)}),
  ToolClass(name='remove', description='Remove information from memory', parameters={'key': FunctionParameter(type='string', description='The key to remove the information from', required=True)}),
  ToolClass(name='respond', description='Send a message to the user', parameters={}),
  ToolClass(name='noop', description='Do nothing', parameters={})],
 'chat_history': [ChatMessage(message='Can you fetch the test_key for me and then respond', created_by='user', created_at=datetime.datetime(2025, 4, 21, 1, 23, 47, 225658, tzinfo=datetime.timezone.utc)),
  ChatMessage(message='The test_key you asked for has the value: test_value. How can I assist you further?', cr

In [110]:
chat_agent.perceive()

18:28:56 - LiteLLM:INFO: utils.py:3085 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
18:28:57 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
18:28:57 - LiteLLM:INFO: cost_calculator.py:636 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO:LiteLLM:selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


KeyError: 'Founding CEO of Agorama'